## KDJ 当日金叉 死叉

In [1]:
import akshare as ak
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from pandas import DataFrame, Series
import pickle

import common
import calculate

In [2]:
file_name = "./txt_lib/stock_code.txt"
with open(file_name, "r") as file:
    stock_list = [line.strip() for line in file if line.strip()]

# # 限制股票数量
# stock_list = stock_list[:150]

# all_cross_dates = process_stocks(stock_list)
all_cross_dates = common.Final_process_KDJ.process_stocks(stock_list)

In [3]:
try:
    folder = Path("./output/golden_output")
    folder.mkdir()
    folder = Path("./output/death_output")
    folder.mkdir()
    folder = Path("./output/操作1.1")
    folder.mkdir()
except Exception as e:
    pass

# 清理两个文件夹
common.Final_file.clean_folder("./output/death_output")
common.Final_file.clean_folder("./output/golden_output")
common.Final_file.clean_folder("./output/操作1.1")
common.Final_file.clean_folder("./output/操作2")

In [4]:
# 选出今日金叉list
today_gold_cross_list = []
for stock_code in all_cross_dates.keys():
    if all_cross_dates[stock_code]["Golden_Cross"]:
        today_gold_cross_list.append(stock_code)

# 选出今日死叉list
today_death_cross_list = []
for stock_code in all_cross_dates.keys():
    if all_cross_dates[stock_code]["Death_Cross"]:
        today_death_cross_list.append(stock_code)

# KDJ日金 + MACD 3 条件 → 操作1.1
to_perfect_stock_list = common.Final_process.process_stocks(
    today_gold_cross_list, mode="MACD_2_condition"
)

Batch 1 completed, memory: 7283.06 MB
Batch 2 completed, memory: 7283.41 MB


In [5]:
# 生成今日金叉 死叉Excel
common.Final_file_KDJ.output_excel(all_cross_dates)

# 生成今日金叉 死叉TXT
print("已生成今日金叉 死叉TXT")
today_death_txt_path = "./output/death_output/today_death_cross.txt"
with open(today_death_txt_path, "w") as file:
    for item in today_death_cross_list:
        file.write(item + "\n")  # 每个元素后加换行符

today_gold_txt_path = "./output/golden_output/today_gold_cross.txt"
with open(today_gold_txt_path, "w") as file:
    for item in today_gold_cross_list:
        file.write(item + "\n")  # 每个元素后加换行符

Excel 文件已生成
已生成今日金叉 死叉TXT


# 按需分流 KDJ日金叉 & KDJ日死叉 代码仓库

In [6]:
def add_capital(input_file: str, output_file: str):
    '''给原始股票代码加上抬头; 600001 → sh600001'''
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            stock_code = line.strip()
            if stock_code.startswith("6"):
                processed_code = f"sh{stock_code}"
            else:
                processed_code = f"sz{stock_code}"
            outfile.write(processed_code + "\n")

    print(f"Processed stock codes saved to {output_file}.")

def copy_yest_txt(input_file: str, output_file: str):
    '''复制一份昨日txt 留档'''
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            stock_code = line.strip()
            outfile.write(stock_code + "\n")
    print(f"Processed stock codes saved to {output_file}.")


dict_filename = "./txt_lib/stock_name.txt"
stock_dict = common.Initialization.generate_stock_dict(dict_filename)

input_file = "./txt_lib/KDJ日金.txt"
output_file = "./txt_lib/yest_KDJ日金.txt"
copy_yest_txt(input_file,output_file)
input_file = "./txt_lib/KDJ日死.txt"
output_file = "./txt_lib/yest_KDJ日死.txt"
copy_yest_txt(input_file, output_file)

# 清理两个文件夹
common.Final_file.clean_folder("./output/KDJ日金")
common.Final_file.clean_folder("./output/KDJ日死")
common.Final_file.clean_folder("./output/KDJ8")

Processed stock codes saved to ./txt_lib/yest_KDJ日金.txt.
Processed stock codes saved to ./txt_lib/yest_KDJ日死.txt.


In [7]:
## 处理 KDJ日金 + 周死叉 → 弃
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日金.txt"
output_file = "./txt_lib/processed_KDJ日金.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    gold_stock_list = [line.strip() for line in file if line.strip()]

gold_to_del_stock_list = []
gold_to_del_week_death = common.Final_process.process_stocks(gold_stock_list, mode="week_death")

## 处理 KDJ日金 + macd < 0 → 弃

gold_macd_below0 = []
gold_macd_below0 = common.Final_process.process_stocks(gold_stock_list, mode="macd < 0")

## 处理 KDJ日金 + J < K → 弃
gold_J_K = []
gold_J_K = common.Final_process.process_stocks(gold_stock_list, mode="J_K")

# 把选出股票加入 to_del 数组中
gold_to_del_stock_list = gold_to_del_week_death + gold_macd_below0 + gold_J_K

Processed stock codes saved to ./txt_lib/processed_KDJ日金.txt.
Batch 1 completed, memory: 7289.94 MB
Batch 1 completed, memory: 7290.04 MB
Batch 1 completed, memory: 7288.36 MB


In [8]:
## 处理 KDJ日死 + 周死叉 → 弃
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日死.txt"
output_file = "./txt_lib/processed_KDJ日死.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    death_stock_list = [line.strip() for line in file if line.strip()]

# 周死叉
death_week_death = []
death_week_death = common.Final_process.process_stocks(death_stock_list, mode="week_death")

# 当日处于金叉阶段
death_day_golden = []
death_day_golden = common.Final_process.process_stocks(
    death_stock_list, mode="day_golden"
)

# 合并要删除的list
death_to_del_stock_list = []
death_to_del_stock_list = death_week_death + death_day_golden

Processed stock codes saved to ./txt_lib/processed_KDJ日死.txt.
Error processing sz300096: 'sz300096'
Batch 1 completed, memory: 7291.54 MB
Batch 2 completed, memory: 7291.99 MB
Error processing sh600831: 'sh600831'
Batch 3 completed, memory: 7289.17 MB
Batch 4 completed, memory: 7288.20 MB
Batch 5 completed, memory: 7289.14 MB
Error processing sz300096: 'sz300096'
Batch 1 completed, memory: 7290.53 MB
Batch 2 completed, memory: 7289.28 MB
Error processing sh600831: 'sh600831'
Batch 3 completed, memory: 7290.67 MB
Batch 4 completed, memory: 7288.50 MB
Batch 5 completed, memory: 7288.09 MB


In [9]:
## 处理 KDJ日金 + 日死 → KDJ日死
gold_to_move_day_death_stock_list = []
gold_to_move_day_death_stock_list = common.Final_process.process_stocks(gold_stock_list, mode="daily_death")

Batch 1 completed, memory: 7290.33 MB


In [10]:
## 处理 KDJ日金 + J线拐头 → KDJ8
gold_to_move_8_stock_list = []
gold_to_move_8_stock_list = common.Final_process.process_stocks(
    gold_stock_list, mode="J_turn_around"
)

Batch 1 completed, memory: 7288.79 MB


In [11]:
## KDJ日死 + Bol: 前次碰上轨，今日碰下轨 → 操作2
death_to_op2_list = []
death_to_op2_list = common.Final_process.process_stocks(death_stock_list, mode="bol_lower")

Error processing sz300096: 'sz300096'
Batch 1 completed, memory: 7290.08 MB
Batch 2 completed, memory: 7288.96 MB
Error processing sh600831: 'sh600831'
Batch 3 completed, memory: 7289.69 MB
Batch 4 completed, memory: 7289.21 MB
Batch 5 completed, memory: 7289.87 MB


In [12]:
## 找出 KDJ日死 接近金叉的股票
death_near_gold_list = []
death_near_gold_list = common.Final_process.process_stocks(
    death_stock_list, mode="kdj_near_gold"
)

Error processing sz300096: 'sz300096'
Batch 1 completed, memory: 7290.70 MB
Batch 2 completed, memory: 7289.84 MB
Error processing sh600831: 'sh600831'
Batch 3 completed, memory: 7288.70 MB
Batch 4 completed, memory: 7291.73 MB
Batch 5 completed, memory: 7290.89 MB


# 图片 Excel生成

In [14]:
try:
    folder = Path("./output")
    folder.mkdir()
except Exception as e:
    pass

try:
    folder = Path("./output/KDJ日金")
    folder.mkdir()
    folder = Path("./output/KDJ日死")
    folder.mkdir()
    folder = Path("./output/KDJ8")
    folder.mkdir()
    folder = Path("./output/操作1.1")
    folder.mkdir()
    folder = Path("./output/操作2")
    folder.mkdir()
except Exception as e:
    pass

In [13]:
# 今日金叉数据
today_gold_txt_path = "./output/golden_output/today_gold_cross.txt"
today_gold_list = []
with open(today_gold_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_gold_list.append(stock_code)

# KDJ日金 文件生成
kdj_res = []
for stock_code in gold_stock_list:
    condition1 = stock_code not in gold_to_del_stock_list # 周死
    condition2 = stock_code not in gold_to_move_day_death_stock_list # 日死
    condition3 = stock_code not in gold_to_move_8_stock_list #拐头
    if condition1 and condition2 and condition3:
        kdj_res.append(stock_code)

# 加入今日金叉数据
for stock_code in today_gold_list:
    if stock_code not in kdj_res:
        kdj_res.append(stock_code)

output_file = './output/KDJ日金/KDJ日金.xlsx'
common.Final_file.output_excel(kdj_res,output_file,condition='日金')
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日金/KDJ日金.xlsx
所有图片已生成！


In [14]:
# 覆写KDJ日金txt
output_file = "./txt_lib/KDJ日金.txt"
with open(output_file, "w") as outfile1:
    for stock_code in kdj_res:
        processed_code = stock_code[2:]
        outfile1.write(processed_code + "\n")

print(f"Processed stock codes saved to {output_file}.")

Processed stock codes saved to ./txt_lib/KDJ日金.txt.


In [15]:
# KDJ8 文件生成

# 读取昨日kdj8 txt
yest_kdj8_list = []
yest_kdj8_path = "./txt_lib/today_kdj8.txt"
with open(yest_kdj8_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        if stock_code.startswith("6"):
            processed_code = f"sh{stock_code}"
        elif stock_code.startswith("0"):
            processed_code = f"sz{stock_code}"
        else:
            processed_code = stock_code
        yest_kdj8_list.append(processed_code)

# 筛出日死代码
kdj8_to_move_day_death_stock_list = []
kdj8_to_move_day_death_stock_list = common.Final_process.process_stocks(
    yest_kdj8_list, mode="daily_death"
)

# 筛出周死代码
kdj8_to_del_stock_list = []
kdj8_to_del_stock_list = common.Final_process.process_stocks(
    yest_kdj8_list, mode="week_death"
)

today_kdj8 = []

for stock_code in yest_kdj8_list:
    condition1 = stock_code not in kdj8_to_del_stock_list # 周死
    condition2 = stock_code not in kdj8_to_move_day_death_stock_list # 日死
    condition3 = stock_code not in gold_to_move_8_stock_list # 今日
    if condition1 and condition2 and condition3:
        today_kdj8.append(stock_code)

today_kdj8 += gold_to_move_8_stock_list

# 最终输出
output_file = "./output/KDJ8/KDJ8.xlsx"
common.Final_file.output_excel(today_kdj8, output_file, condition="8")
common.Final_file.mk_pic(output_file)  # 生成所有图片

# 生成今日 KDJ8 TXT
today_kdj8_txt_path = "./txt_lib/today_kdj8.txt"
with open(today_kdj8_txt_path, "w") as file:
    for item in today_kdj8:
        file.write(item + "\n")  # 每个元素后加换行符

print("今日KDJ8 txt文件已生成。")

Batch 1 completed, memory: 7295.62 MB
Batch 2 completed, memory: 7296.47 MB
Batch 1 completed, memory: 7296.06 MB
Batch 2 completed, memory: 7296.57 MB
Excel 文件已生成: ./output/KDJ8/KDJ8.xlsx
所有图片已生成！
今日KDJ8 txt文件已生成。


In [16]:
# KDJ日死 文件生成
kdj_death = []

# 今日死叉数据
today_death_txt_path = "./output/death_output/today_death_cross.txt"
today_death_list = []
with open(today_death_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_death_list.append(stock_code)

for stock_code in death_stock_list:
    condition1 = stock_code not in death_to_del_stock_list
    # condition2 = stock_code not in death_to_op2_list
    if condition1:
        kdj_death.append(stock_code)

# 加入今日死叉数据
for stock_code in today_death_list:
    if stock_code not in kdj_death:
        kdj_death.append(stock_code)
        
# 加入今日kdj8日死数据
for stock_code in kdj8_to_move_day_death_stock_list:
    if stock_code not in kdj_death:
        kdj_death.append(stock_code)

output_file = "./output/KDJ日死/KDJ日死.xlsx"
common.Final_file.output_excel(kdj_death, output_file, condition="日死")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日死/KDJ日死.xlsx
所有图片已生成！


## 操作2

In [17]:
# KDJ日死 + 今日是第二根阴线 + 日MACD > 0 + '第三日'创最近十日新高 + 最近两天不能碰中轨 → 操作2
op2_3 = []
op2_3 = common.Final_process.process_stocks(kdj_death, mode="op2_3")

Error processing sz300096: 'sz300096'
Error processing sh600831: 'sh600831'
Batch 1 completed, memory: 7296.86 MB


In [ ]:
# 输出操作2 Excel 图片
op2 = []
op2 = death_to_op2_list + death_near_gold_list + op2_3

# 当日没有碰上轨
final_list = []
final_list = common.Final_process.process_stocks(op2, mode="not_touch_upper")

# 利润 > 0 -> 保留
# 利润 < 0 + 周&月macd dif > 0 -> 保留
profit_list = []
profit_list = common.Final_process.process_stocks(final_list, mode="profit_check")

output_file = "./output/操作2/操作2.xlsx"
common.Final_file.output_excel(profit_list, output_file, condition="bol")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/操作2/操作2.xlsx
所有图片已生成！


In [19]:
# 覆写KDJ日死txt
output_file = "./txt_lib/KDJ日死.txt"
with open(output_file, "w") as outfile1:
    for stock_code in kdj_death:
        processed_code = stock_code[2:]
        outfile1.write(processed_code + "\n")

print(f"Processed stock codes saved to {output_file}.")

Processed stock codes saved to ./txt_lib/KDJ日死.txt.


## 操作1.1

In [20]:
# KDJ金叉
# 前次碰上轨 + 今日碰中轨 + 昨日最高价 > 今日最高价 → 操作1.1

gold_to_1_1_list = []
gold_to_1_1_list = common.Final_process.process_stocks(kdj_res, mode="bol")

Batch 1 completed, memory: 7293.26 MB
Batch 2 completed, memory: 7294.70 MB


In [21]:
# 处理 KDJ日金 + 阴线 + 周kdj → 操作1.1
# 判断是否今日是第一根阴线 + 周kdj金叉三周内
# + 昨日阳线碰Bol上轨 + 今日最高价小于昨日最高价

op1_1_cond3 = []
op1_1_cond3 = common.Final_process.process_stocks(kdj_res, mode="阴线_周kdj")

Batch 1 completed, memory: 7297.12 MB
Batch 2 completed, memory: 7294.59 MB


In [22]:
# 输出操作1.1 Excel 图片
op1_1 = []
op1_1 = to_perfect_stock_list + gold_to_1_1_list + op1_1_cond3
output_file = "./output/操作1.1/操作1.1.xlsx"

common.Final_file.output_excel(op1_1, output_file, condition="bol")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/操作1.1/操作1.1.xlsx
所有图片已生成！


## TEST

In [25]:
from importlib import reload
reload(calculate)
reload(common)

<module 'common' from 'e:\\GRADUATE\\project\\stock_info\\common.py'>

In [3]:
stock_code = "sz000913"
current_date = datetime.now()

df = ak.stock_zh_a_daily(
    symbol=stock_code,
    start_date="2015-01-01",
    end_date=current_date.strftime("%Y-%m-%d"),
)

In [4]:
week_data = calculate.KDJ.calculate_weekly_kdj(df)

In [5]:
week_data.tail(7)

,open,high,low,close,K,D,J
date,,,,,,,
2025-05-16,15.90,16.20,15.69,15.87,27.123,27.784,25.801
2025-05-23,15.96,16.45,15.56,15.56,27.688,27.752,27.560
2025-05-30,15.61,15.72,14.95,14.98,23.425,26.310,17.655
2025-06-06,14.88,16.23,14.71,15.80,32.050,28.223,39.704
2025-06-13,15.79,16.14,15.11,15.13,26.879,27.775,25.087
2025-06-20,15.21,15.50,14.83,14.87,20.019,25.190,9.677
2025-06-30,14.74,15.14,14.55,14.92,19.837,23.406,12.699


In [8]:
week_data["D"].iloc[-1] > week_data["J"].iloc[-1]

True